In [1]:
#Import packages
#from polygon.rest import RESTClient
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import numpy as np

#Import API keys from file
polygon_api_key = open("polygon_io_api_key.txt").read()
alphavantage_api_key = open("alphavantage_api_key.txt").read()

#Ticker
ticker = "kr".upper()

# Calls the API to get historical stock data
stock_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={alphavantage_api_key}'
stock_r = requests.get(stock_url)
stock_data = stock_r.json()

earnings_url = f"https://api.polygon.io/benzinga/v1/earnings?ticker={ticker}&limit=100&sort=date.desc&apiKey={polygon_api_key}&date_status=confirmed"
earnings_r = requests.get(earnings_url)
earnings_data = earnings_r.json()

In [ ]:

##############
# Stock Data #
##############

# Extract the time series data
time_series = stock_data['Time Series (Daily)']

# Convert the time series data into a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Convert the columns to numeric
df = df.apply(pd.to_numeric)

df = df.reset_index().rename(columns={'index': 'tradingDate'})

#Earnings
df_len = []
for i in range(len(earnings_data["results"])):
    df_len.append(i)

benzinga_earnings = pd.DataFrame(earnings_data["results"], index=df_len)
df_column_drop = ['actual_eps','estimated_eps','previous_eps','eps_surprise_percent','revenue_surprise_percent','currency','date_status','benzinga_id','importance','company_name','estimated_revenue','previous_revenue','last_updated','eps_surprise','eps_method','actual_revenue','revenue_surprise','revenue_method',]
benzinga_earnings.drop(df_column_drop, axis='columns', inplace=True)

noon = dt.time(12,0,0)
market_periods = []

benzinga_earnings.loc[:, "time"] = pd.to_datetime(benzinga_earnings["time"], format='%H:%M:%S').dt.time

for time in benzinga_earnings["time"]:
    if time >= noon:
        market_periods.append('after-market')
    else:
        market_periods.append('pre-market')

benzinga_earnings.loc[:,'market_period'] = market_periods
benzinga_earnings.drop('time',axis="columns", inplace=True)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['market_period'] == 'after-market':
       return df[df["tradingDate"] > row['date']].min(axis=0)["tradingDate"]
   elif row['market_period'] == 'pre-market':
       return df[df["tradingDate"] < row['date']].max(axis=0)["tradingDate"]
   else:
       return row['date']

benzinga_earnings['date'] = pd.to_datetime(benzinga_earnings['date'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
benzinga_earnings['adjustedDate'] = benzinga_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(benzinga_earnings, df, left_on='date', right_on="tradingDate", how='left')
columns_to_drop = ['tradingDate','5. volume']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   '1. open': 'report open',
   '2. high': 'report high',
   '3. low': 'report low',
   '4. close': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="tradingDate", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   '1. open': 'adjusted open',
   '2. high': 'adjusted high',
   '3. low': 'adjusted low',
   '4. close': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df.sort_values(by="percent_diff", ascending=False)

,fiscal_period,fiscal_year,ticker,date,notes,market_period,adjustedDate,report open,report high,report low,report close,adjusted open,adjusted high,adjusted low,adjusted close,diff,percent_diff
31,Q3,2017,KR,2017-11-30,NaN,pre-market,2017-11-29,27.320,27.7000,25.5300,25.86,23.65,24.4000,23.6200,24.38,4.08,0.173
14,Q4,2021,KR,2022-03-03,NaN,pre-market,2022-03-02,52.250,55.5000,52.2500,55.10,48.09,49.4850,48.0900,49.37,7.41,0.154
29,Q1,2018,KR,2018-06-21,NaN,pre-market,2018-06-20,28.290,29.5000,27.7500,28.73,26.22,26.3900,25.8400,26.18,3.66,0.142
15,Q3,2021,KR,2021-12-02,NaN,pre-market,2021-12-01,42.460,45.6800,41.7300,44.65,41.53,41.7800,40.1800,40.21,5.50,0.137
6,Q4,2023,KR,2024-03-07,NaN,pre-market,2024-03-06,52.250,55.5350,51.7274,55.48,49.71,50.6500,49.4347,50.49,6.10,0.123
1,Q1,2025,KR,2025-06-20,NaN,pre-market,2025-06-18,68.205,72.2300,67.5000,71.97,66.00,66.1800,65.2150,65.52,7.02,0.108
40,Q2,2015,KR,2015-09-11,NaN,pre-market,2015-09-10,37.410,37.7500,36.0900,37.29,34.43,35.6100,34.3100,35.40,3.44,0.100
4,Q2,2024,KR,2024-09-12,NaN,pre-market,2024-09-11,52.380,55.2400,51.5200,55.20,51.48,51.7700,50.6900,51.50,4.55,0.090
10,Q4,2022,KR,2023-03-02,NaN,pre-market,2023-03-01,45.010,46.6200,44.2100,45.73,42.96,43.4300,42.8150,43.38,3.80,0.089
42,Q4,2014,KR,2015-03-05,NaN,pre-market,2015-03-04,73.500,74.9700,72.6000,74.31,69.48,69.6840,69.0800,69.65,5.89,0.085
